In [1]:
embedding = "tuned_embedding_1.5M"
root = "/Users/LM3C3S/PycharmProjects/personality_prediction/outputs"

In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/Users/LM3C3S/PycharmProjects/personality_prediction'

In [3]:
import pandas as pd
import os
import pickle
import numpy as np
from fnn import fnn_model, data_loader

#### LOAD PERF TUNED EMBEDDING 1.5M

In [ ]:
df_perf = pd.read_excel("/Users/LM3C3S/PycharmProjects/personality_prediction/outputs/tuned_embedding_1.5M/performances_coerence.xlsx").iloc[0:,1:]

In [ ]:
df_perf = df_perf[df_perf["distance"]==4]

In [ ]:
df_perf.groupby("fold").max("best_r2")

#### PREDICTIONS

In [4]:
dl = data_loader.Data_Loader(distance=None, create_data=True, create_tree = True, standardize=True, embedding_name=embedding, k_folds=None)

_______ OCEAN VOCABULARY CREATED
____ PREPROCESSING DONE
... Creating the embedding
____ PREPROCESSING DONE
... Creating the embedding
____ PREPROCESSING DONE
... Creating the embedding
____ PREPROCESSING DONE
... Creating the embedding
____ PREPROCESSING DONE
... Creating the embedding


In [5]:
models = [fnn_model.simpleModel() for i in range(0,5)]

##### fit

In [ ]:
for i in range(0,5):
    print(i)
    models[i].fit_predict(dl.data[i].train_inputs, dl.data[i].train_outputs[i], epochs=300, verbose=0)

In [ ]:
weights = [m.model.get_weights() for m in models]

In [ ]:
import pickle
with open(os.path.join(root,embedding,"weights.pickle"), "wb") as f:
    pickle.dump(weights, f)

##### load weights

In [6]:
import pickle
with open(os.path.join(root,embedding,"weights.pickle"), "rb") as f:
    weights = pickle.load(f)
for i in range(0,5):
    models[i].model.set_weights(weights[i])

##### predict

In [7]:
predictions = [models[i].model.predict(dl.data[i].inputs_neig) for i in range(0,5)]
predictions = [np.reshape(p, len(p)) for p in predictions]

In [8]:
dl2 = data_loader.Data_Loader(oceans = [0], distance=None, create_data=True, create_tree = True, standardize=False, embedding_name="tuned_embedding_1.5M", k_folds=None).data[0]

_______ OCEAN VOCABULARY CREATED
____ PREPROCESSING DONE
... Creating the embedding


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean = True, with_std = True)

In [10]:
scaler.fit(np.transpose(dl2.train_outputs))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [11]:
predictions = scaler.inverse_transform(np.transpose(predictions))

In [14]:
df_predictions = pd.DataFrame(predictions, columns=["O","C","E","A","N"])
df_predictions["word"] = dl.data[0].words_not_ocean

In [ ]:
df_predictions

In [12]:
from scipy.spatial.distance import euclidean as dist

In [15]:
for c in ["O","C","E","A","N"]:
    df_predictions[c] = df_predictions[c].apply(lambda x : min(1,x))
    df_predictions[c] = df_predictions[c].apply(lambda x : max(-1,x))

In [16]:
df_predictions["cont"] = list(range(0,len(df_predictions)))

In [17]:
df_predictions["MK_O+"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([1,0,0,0,0])), axis=1, raw=True)
df_predictions["MK_O-"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([-1,0,0,0,0])), axis=1, raw=True)
df_predictions["MK_C+"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,1,0,0,0])), axis=1, raw=True)
df_predictions["MK_C-"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,-1,0,0,0])), axis=1, raw=True)
df_predictions["MK_E+"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,0,1,0,0])), axis=1, raw=True)
df_predictions["MK_E-"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,0,-1,0,0])), axis=1, raw=True)
df_predictions["MK_A+"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,0,0,1,0])), axis=1, raw=True)
df_predictions["MK_A-"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,0,0,-1,0])), axis=1, raw=True)
df_predictions["MK_N+"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,0,0,0,1])), axis=1, raw=True)
df_predictions["MK_N-"] = df_predictions[["O","C","E","A","N"]].apply(lambda x : 1-dist(x, np.asarray([0,0,0,0,-1])), axis=1, raw=True)

In [18]:
marker_index = ["MK_O+", "MK_O-", "MK_C+", "MK_C-", "MK_E+", "MK_E-", "MK_A+", "MK_A-", "MK_N+", "MK_N-"]
ocean = {"O":0,"C":1,"E":2,"A":3,"N":4}
for mk in marker_index:
    oc = mk[3:4]
    cont_oc = ocean[oc]
    df_predictions[mk] = df_predictions[["cont",oc,mk]].apply(lambda x : x[2] if dl.data[cont_oc].is_significant_term(int(x[0]), x[1], cont_oc) else -100, raw=True, axis=1)

... Creating the known tree
... Querying the known tree
... Creating the known tree
... Querying the known tree
... Creating the known tree
... Querying the known tree
... Creating the known tree
... Querying the known tree
... Creating the known tree
... Querying the known tree


In [19]:
df_predictions

,O,C,E,A,N,word,cont,MK_O+,MK_O-,MK_C+,MK_C-,MK_E+,MK_E-,MK_A+,MK_A-,MK_N+,MK_N-
0,-0.007608,0.013631,-0.035440,0.134971,0.007392,!,0,-0.017343,-0.002275,0.003752,-0.023246,-0.044342,0.025889,0.134073,-0.135656,-0.002491,-0.017131
1,-0.003214,-0.006326,0.014983,0.117086,-0.042307,not,1,-0.011040,-0.004662,-0.014113,-0.001560,0.007122,-0.022615,0.115917,-0.118009,-0.048994,0.035034
2,0.026095,-0.017712,-0.008838,0.014989,0.146189,get,2,0.014871,-0.036754,-0.028636,0.006398,-0.019973,-0.002493,0.003661,-0.025985,0.145430,-0.146755
3,0.008936,-0.016266,0.121332,0.102384,0.011195,good,3,-0.003893,-0.021541,-0.028691,0.003435,0.115121,-0.126205,0.093962,-0.109253,-0.001641,-0.023749
4,0.057819,-0.044941,-0.090218,0.043459,-0.060051,place,4,0.049548,-0.065193,-0.053033,0.036093,-0.095186,0.084271,0.034558,-0.051625,-0.067286,0.051900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59476,0.110391,0.191497,-0.125226,0.063852,-0.264790,submersion,59476,0.041905,-0.165981,-100.000000,-100.000000,-0.178635,0.057515,-0.005497,-0.125360,-0.291631,0.219520
59477,-0.065228,-0.044693,-0.232052,0.033362,-0.218086,cool-down,59477,-0.113207,0.010900,-0.094605,-0.009648,-0.254145,0.197086,-0.020808,-0.084205,-100.000000,-100.000000
59478,-0.453561,0.208981,-0.000778,0.170775,0.211875,popolo,59478,-0.493509,0.354768,-100.000000,-100.000000,-0.151087,-0.149735,-100.000000,-100.000000,0.051477,-0.321816
59479,-0.113998,-0.069982,0.294792,0.247245,-0.027642,tusk,59479,-0.180968,0.031138,-0.143089,-0.013275,-100.000000,-100.000000,-100.000000,-100.000000,-0.105429,-0.054232


In [25]:
df_predictions.to_excel(os.path.join(root,embedding,"predictions.xlsx"), index=False)

In [24]:
char = "N+"
df_predictions.sort_values("MK_"+char, ascending=False).head(50)[["word","MK_"+char]]

,word,MK_N+
57455,margolies,0.583118
42212,espo,0.582306
50222,così,0.549766
34114,raum,0.540310
33124,alessandra,0.537262
25941,kem,0.536028
11939,digestive,0.523073
41858,rheumatoid,0.513668
49486,romina,0.513081
44080,intractable,0.510297


In [ ]:
char = "O"
df_predictions.sort_values(char, ascending=False).head(50)[["word", char]]